##### Imports
Veuillez lancer la cellule suivante pour importer les librairies utilisées dans ce rapport

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from IPython.display import display

from sklearn.datasets import fetch_openml

from sklearn.model_selection import train_test_split

In [2]:
df = sklearn.datasets.fetch_openml(name="fps-in-video-games")

In [36]:
bc_df = pd.DataFrame(df.data, columns=df.feature_names)

bc_df_tab = 

#print("X_train shape: {}".format(bc_df['GpuMemoryBus'][59]))
print(np.isnan(bc_df['GpuMemoryBus']))

TypeError: Object with dtype category cannot perform the numpy op isnan